In [2]:
car = {
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}

x = list(car.items())

for i in range(len(x)):
	a,b = x[i]
	print(a,b)


brand Ford
model Mustang
year 1964


In [2]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque
rep_count=0

# Initialize Pose Detection
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_draw = mp.solutions.drawing_utils

# Video Capture
cap = cv2.VideoCapture(1)

# Rep Counting Variables
rep_count = 0
left_reached = False
right_reached = False
angle_threshold = 30  # Angle threshold for left & right rotation

# Moving Average Filter Variables
window_size = 5  # Number of frames to average
angle_history = deque(maxlen=window_size)  # Queue to store past angles

def calculate_angle(a, b, c):
    """Calculate the angle between three points."""
    a = np.array(a)  # Shoulder
    b = np.array(b)  # Hip Center
    c = np.array(c)  # Opposite Shoulder

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))

    return np.degrees(angle) - 90  # Adjust so neutral position is ~0°

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(rgb_frame)

    if result.pose_landmarks:
        mp_draw.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        landmarks = result.pose_landmarks.landmark
        
        # Get landmark positions
        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, 
                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, 
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
        hip_center = [(landmarks[mp_pose.PoseLandmark.LEFT_HIP].x + landmarks[mp_pose.PoseLandmark.RIGHT_HIP].x) / 2, 
                      (landmarks[mp_pose.PoseLandmark.LEFT_HIP].y + landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y) / 2]

        # Calculate rotation angle
        raw_rotation_angle = calculate_angle(left_shoulder, hip_center, right_shoulder)

        # Apply Moving Average Filter
        angle_history.append(raw_rotation_angle)  # Store latest value
        smooth_rotation_angle = np.mean(angle_history)  # Compute moving average

        # Rep counting logic
        left_reached, right_reached, rep_count

        if smooth_rotation_angle > 30:  # Rotated left
            left_reached = True
        if smooth_rotation_angle < -30:  # Rotated right
            right_reached = True

        # Count rep when both sides have been reached sequentially
        if left_reached and right_reached:
            rep_count += 1
            left_reached = False
            right_reached = False

        # Display rotation angle & reps
        cv2.putText(frame, f"Raw Angle: {int(raw_rotation_angle)}", (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
        cv2.putText(frame, f"Smooth Angle: {int(smooth_rotation_angle)}", (50, 80), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.putText(frame, f"Reps: {rep_count}", (50, 120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    cv2.imshow("Russian Twist Tracker", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time,datetime

mp_drawing=mp.solutions.drawing_utils #for drwaing 
mp_pose=mp.solutions.pose #for pose extimator
mp_hand=mp.solutions.hands #for pose extimator



def calculate_angle(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)

    radians=np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)

    if angle>180.0:
        angle=360-angle

    return angle


def calculate_distance(a,b):
    a=np.array(a)
    b=np.array(b)

    distance=np.sqrt(np.sum((a-b)**2))
    return distance 

def mind_point_finder(a,b):
    a=list(a)
    b=list(b)
    result={}
    if len(a) == len(b):
        for i in range(0,len(a)):
            result[i]=(a[i]+b[i])/2
    return list(result.values()) 

counter=0
step =None
full_time=0.0

cap=cv2.VideoCapture(1)
#setup mediapipe instances.
with mp_pose.Pose(min_detection_confidence=0.9,min_tracking_confidence=0.9) as pose:
    while cap.isOpened():
        ret,frame=cap.read()
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False    #adjusting the COlour for psoe -mediapipe

        result=pose.process(image)  #passing the image to psoe

        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)  # readhusitng for rendering to window

        #extract landmark
        try:
            
            landmarks=result.pose_landmarks.landmark

            right_elbow=[landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist=[landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]


            left_elbow=[landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x ,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist=[landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x ,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            mid_point_right=mind_point_finder(a=right_elbow,b=right_wrist)
            mid_point_left=mind_point_finder(a=left_wrist,b=left_elbow)

            distance_between_mid_point=calculate_distance(a=mid_point_left,b=mid_point_right)

            if distance_between_mid_point <= 0.06:
                counter+=1
                
                         
            if counter == 10:
                break
                

            cv2.putText(image,str(round(distance_between_mid_point,3)),
                        tuple(np.multiply(right_shoulder, [640, 480]).astype(int)),
                              cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
            cv2.putText(image,str(counter),(15,60),
                    cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
            
    
        except:
            pass
     

        #rendering detection.
        mp_drawing.draw_landmarks(image,result.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        cv2.imshow("Mediapipe Feed",image)

        
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break
    
    

    
    cap.release()
    cv2.destroyAllWindows()



In [3]:
import cv2
import mediapipe as mp
import numpy as np
import time,datetime

mp_drawing=mp.solutions.drawing_utils #for drwaing 
mp_pose=mp.solutions.pose #for pose extimator
mp_hand=mp.solutions.hands #for pose extimator



def calculate_angle(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)

    radians=np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)

    if angle>180.0:
        angle=360-angle

    return angle


def calculate_distance(a,b):
    a=np.array(a)
    b=np.array(b)

    distance=np.sqrt(np.sum((a-b)**2))
    return distance 

def mind_point_finder(a,b):
    a=list(a)
    b=list(b)
    result={}
    if len(a) == len(b):
        for i in range(0,len(a)):
            result[i]=(a[i]+b[i])/2
    return list(result.values()) 

counter=0
step =None
full_time=0.0

cap=cv2.VideoCapture(1)
#setup mediapipe instances.
with mp_pose.Pose(min_detection_confidence=0.9,min_tracking_confidence=0.9) as pose:
    while cap.isOpened():
        ret,frame=cap.read()
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False    #adjusting the COlour for psoe -mediapipe

        result=pose.process(image)  #passing the image to psoe

        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)  # readhusitng for rendering to window

        #extract landmark
        try:
            
            landmarks=result.pose_landmarks.landmark

            right_elbow=[landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist=[landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]


            left_elbow=[landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x ,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            left_wrist=[landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x ,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            dist_wrist=calculate_distance(right_wrist,left_wrist)
                

            cv2.putText(image,str(round(dist_wrist,3)),
                        tuple(np.multiply(right_wrist, [640, 480]).astype(int)),
                              cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
            cv2.putText(image,str(counter),(15,60),
                    cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
            
    
        except:
            pass
     

        #rendering detection.
        mp_drawing.draw_landmarks(image,result.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        cv2.imshow("Mediapipe Feed",image)

        
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break
    
    

    
    cap.release()
    cv2.destroyAllWindows()



In [5]:
import cv2
import mediapipe as mp
import numpy as np
import time,datetime

mp_drawing=mp.solutions.drawing_utils #for drwaing 
mp_pose=mp.solutions.pose #for pose extimator
mp_hand=mp.solutions.hands #for pose extimator



def calculate_angle(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)

    radians=np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)

    if angle>180.0:
        angle=360-angle

    return angle


def calculate_distance(a,b):
    a=np.array(a)
    b=np.array(b)

    distance=np.sqrt(np.sum((a-b)**2))
    return distance 

def mind_point_finder(a,b):
    a=list(a)
    b=list(b)
    result={}
    if len(a) == len(b):
        for i in range(0,len(a)):
            result[i]=(a[i]+b[i])/2
    return list(result.values()) 

counter=0
step =None
full_time=0.0

cap=cv2.VideoCapture(1)
#setup mediapipe instances.
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame=cap.read()
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False    #adjusting the COlour for psoe -mediapipe

        result=pose.process(image)  #passing the image to psoe

        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)  # readhusitng for rendering to window

        #extract landmark
        try:
            
            landmarks=result.pose_landmarks.landmark

            right_ankle=[landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_knee=[landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_hip=[landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_shoulder=[landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            left_ankle=[landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x ,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            left_knee=[landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x ,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_hip=[landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x ,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            

            rigth_knee_angle=calculate_angle(right_ankle,right_knee,right_hip)
            left_knee_angle=calculate_angle(left_ankle,left_knee,left_hip)
            shoulder_angle=calculate_angle(right_shoulder,right_hip,right_knee)

            if rigth_knee_angle  <= 90 and left_knee_angle <= 90 and shoulder_angle  <= 120:
                step="DOWN"
            if step=="DOWN" and  rigth_knee_angle >= 175 and left_knee_angle >= 175 and shoulder_angle > 175:
                step="UP"
                counter+=1
                
                


                

            cv2.putText(image,str(round(rigth_knee_angle,3)),
                        tuple(np.multiply(right_knee, [640, 480]).astype(int)),
                              cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
            
            cv2.putText(image,str(round(left_knee_angle,3)),
                        tuple(np.multiply(left_knee, [640, 480]).astype(int)),
                              cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
            
            cv2.putText(image,str(round(shoulder_angle,3)),
                        tuple(np.multiply(right_shoulder, [640, 480]).astype(int)),
                              cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
            
            cv2.putText(image,f"Counter: {str(counter)}/ Step : {str(step)} ",(15,60),
                    cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
            
    
        except:
            pass
     

        #rendering detection.
        mp_drawing.draw_landmarks(image,result.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        cv2.imshow("Mediapipe Feed",image)

        
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break
    
    

    print(f"COUNT : {counter}")
    cap.release()
    cv2.destroyAllWindows()



COUNT : 0


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time,datetime

mp_drawing=mp.solutions.drawing_utils #for drwaing 
mp_pose=mp.solutions.pose #for pose extimator
mp_hand=mp.solutions.hands #for pose extimator



def calculate_angle(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)

    radians=np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)

    if angle>180.0:
        angle=360-angle

    return angle


def calculate_distance(a,b):
    a=np.array(a)
    b=np.array(b)

    distance=np.sqrt(np.sum((a-b)**2))
    return distance 

def mind_point_finder(a,b):
    a=list(a)
    b=list(b)
    result={}
    if len(a) == len(b):
        for i in range(0,len(a)):
            result[i]=(a[i]+b[i])/2
    return list(result.values()) 

counter=0
step =None
full_time=0.0

cap=cv2.VideoCapture(0)
#setup mediapipe instances.
with mp_pose.Pose(min_detection_confidence=0.9,min_tracking_confidence=0.9) as pose:
    while cap.isOpened():
        ret,frame=cap.read()
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False    #adjusting the COlour for psoe -mediapipe

        result=pose.process(image)  #passing the image to psoe

        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)  # readhusitng for rendering to window

        #extract landmark
        try:
            
            landmarks=result.pose_landmarks.landmark

            right_ankle=[landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_knee=[landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_hip=[landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_shoulder=[landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_wrist=[landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_heel=[landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y]
            left_

            hip_angle=calculate_angle(right_knee,right_hip,right_shoulder)
            knee_angle=calculate_angle(right_ankle,right_knee,right_hip)

            if hip_angle <= 120 and knee_angle <= 30:
                step="DOWN"
            if step=="DOWN" and hip_angle >=175 and knee_angle >= 40:
                step="UP"
                counter+=1
            
          


                

            cv2.putText(image,str(round(hip_angle,3)),
                        tuple(np.multiply(right_hip, [640, 480]).astype(int)),
                              cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
            
            cv2.putText(image,str(round(knee_angle,3)),
                        tuple(np.multiply(right_knee, [640, 480]).astype(int)),
                              cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
            
            cv2.putText(image,f"Counter: {str(counter)}/ Step : {str(step)} ",(15,60),
                    cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
    
        except:
            pass
     

        #rendering detection.
        mp_drawing.draw_landmarks(image,result.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        cv2.imshow("Mediapipe Feed",image)

        
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break
    
    


    cap.release()
    cv2.destroyAllWindows()



In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time,datetime
from src.pose_estimator.intensed_upper_body_push import UpperBodyPush



warm=UpperBodyPush.warmup

mp_drawing=mp.solutions.drawing_utils #for drwaing 
mp_pose=mp.solutions.pose #for pose extimator
mp_hand=mp.solutions.hands #for pose extimator



def calculate_angle(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)

    radians=np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)

    if angle>180.0:
        angle=360-angle

    return angle


def calculate_distance(a,b):
    a=np.array(a)
    b=np.array(b)

    distance=np.sqrt(np.sum((a-b)**2))
    return distance 

def mind_point_finder(a,b):
    a=list(a)
    b=list(b)
    result={}
    if len(a) == len(b):
        for i in range(0,len(a)):
            result[i]=(a[i]+b[i])/2
    return list(result.values()) 

counter=0
step =None
full_time=0.0

cap=cv2.VideoCapture(0)
#setup mediapipe instances.
with mp_pose.Pose(min_detection_confidence=0.9,min_tracking_confidence=0.9) as pose:
    while cap.isOpened():
        ret,frame=cap.read()
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False    #adjusting the COlour for psoe -mediapipe

        result=pose.process(image)  #passing the image to psoe

        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)  # readhusitng for rendering to window

        #extract landmark
        try:
            
            landmarks=result.pose_landmarks.landmark         
            counter,step=warm(landmarks=landmarks)               

            
            cv2.putText(image,f"Counter: {str(counter)}/ Step : {str(step)} ",(15,60),
                    cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
    
        except:
            pass
     

        #rendering detection.
        mp_drawing.draw_landmarks(image,result.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        cv2.imshow("Mediapipe Feed",image)

        
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break
    
    


    cap.release()
    cv2.destroyAllWindows()



ModuleNotFoundError: No module named 'src'

In [7]:
import cv2
import mediapipe as mp
import numpy as np
import time,datetime

mp_drawing=mp.solutions.drawing_utils #for drwaing 
mp_pose=mp.solutions.pose #for pose extimator
mp_hand=mp.solutions.hands #for pose extimator



def calculate_angle(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)

    radians=np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)

    if angle>180.0:
        angle=360-angle

    return angle


def calculate_distance(a,b):
    a=np.array(a)
    b=np.array(b)

    distance=np.sqrt(np.sum((a-b)**2))
    return distance 

def mind_point_finder(a,b):
    a=list(a)
    b=list(b)
    result={}
    if len(a) == len(b):
        for i in range(0,len(a)):
            result[i]=(a[i]+b[i])/2
    return list(result.values()) 

counter=0
step =None
full_time=0.0

cap=cv2.VideoCapture("D:\gitfit-tracker\IMG\images.jpg")
#setup mediapipe instances.
with mp_pose.Pose(min_detection_confidence=0.9,min_tracking_confidence=0.9) as pose:
    while cap.isOpened():
        ret,frame=cap.read()
        #image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False    #adjusting the COlour for psoe -mediapipe

        result=pose.process(image)  #passing the image to psoe

        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)  # readhusitng for rendering to window

        #extract landmark
        try:
            
            landmarks=result.pose_landmarks.landmark

            right_elbow=[landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_wrist=[landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            right_shoulder=[landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            elbow=calculate_angle(right_wrist,right_elbow,right_shoulder)

            cv2.putText(image,str(round(elbow,3)),
                        tuple(np.multiply(right_shoulder, [640, 480]).astype(int)),
                              cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
            cv2.putText(image,str(counter),(15,60),
                    cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
            
    
        except:
            pass
     

        #rendering detection.
        mp_drawing.draw_landmarks(frame,result.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        cv2.imshow("Mediapipe Feed",image)

        
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break
    
    

    
    cap.release()
    cv2.destroyAllWindows()



<>:46: SyntaxWarning: invalid escape sequence '\g'
<>:46: SyntaxWarning: invalid escape sequence '\g'
C:\Users\tsaja\AppData\Local\Temp\ipykernel_10316\1376871878.py:46: SyntaxWarning: invalid escape sequence '\g'
  cap=cv2.VideoCapture("D:\gitfit-tracker\IMG\images.jpg")


In [23]:
import cv2
import mediapipe as mp
import numpy as np
import time,datetime

mp_drawing=mp.solutions.drawing_utils #for drwaing 
mp_pose=mp.solutions.pose #for pose extimator
mp_hand=mp.solutions.hands #for pose extimator



def calculate_angle(a,b,c):
    a=np.array(a)
    b=np.array(b)
    c=np.array(c)

    radians=np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)

    if angle>180.0:
        angle=360-angle

    return angle


def calculate_distance(a,b):
    a=np.array(a)
    b=np.array(b)

    distance=np.sqrt(np.sum((a-b)**2))
    return distance 

def mind_point_finder(a,b):
    a=list(a)
    b=list(b)
    result={}
    if len(a) == len(b):
        for i in range(0,len(a)):
            result[i]=(a[i]+b[i])/2
    return list(result.values()) 

counter=0
step =None
workout_time=0.0

cap=cv2.VideoCapture(1)
#setup mediapipe instances.
with mp_pose.Pose(min_detection_confidence=0.9,min_tracking_confidence=0.9) as pose:
    while cap.isOpened():
        ret,frame=cap.read()
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False    #adjusting the COlour for psoe -mediapipe

        result=pose.process(image)  #passing the image to psoe

        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        #extract landmark
        try:
            
            landmarks=result.pose_landmarks.landmark

            right_elbow=[landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            right_shoulder=[landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_wrist=[landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x ,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]





            right_shoulder_angle=calculate_angle(a=right_wrist,b=right_elbow,c=right_shoulder)







            cv2.putText(image,str(round(right_shoulder_angle,3)),
                        tuple(np.multiply(right_shoulder, [640, 480]).astype(int)),
                              cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
          
            cv2.putText(image,str(round(left_knee_angle,3)),
                        tuple(np.multiply(left_knee, [640, 480]).astype(int)),
                              cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
            
            cv2.putText(image,str(step),(15,60),
                    cv2.FONT_HERSHEY_COMPLEX,1,(225,225,225),2,cv2.LINE_AA)
    
        except:
            pass
     

        #rendering detection.
        mp_drawing.draw_landmarks(image,result.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))
        
        cv2.imshow("Mediapipe Feed",image)

        
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break
    
    

    print(f"Total Workout time is :{workout_time}")
    cap.release()
    cv2.destroyAllWindows()



Total Workout time is :0.0


In [1]:
workout_set_reps={"Squart":{"reps":12,
                            "set":3},
                    "Push Up":{"reps":12,
                               "sets":3}}



In [6]:
(workout_set_reps["Squart"]["reps"]) * 2/3

8.0

In [2]:
import yaml
fullbody_moderate_intensity={"Standard PushUp":{"reps":12,"set":3,"rest_time":45},
                            "Standard Squart":{"reps":12,"sets":3,"rest_time":45},
                            "Hollow Body Hold":{"reps":30,"set":3,"rest_time":45},
                            "Inverted Row":{"reps":12,"set":3,"rest_time":45},
                            "Plank Hold":{"reps":30,"set":3,"rest_time":30},
                            "Forward Lunge":{"reps":12,"set":3,"rest_time":45}}



ymal_str=yaml.dump(fullbody_moderate_intensity,default_flow_style=False,sort_keys=False)
print(ymal_str)

Standard PushUp:
  reps: 12
  set: 3
  rest_time: 45
Standard Squart:
  reps: 12
  sets: 3
  rest_time: 45
Hollow Body Hold:
  reps: 30
  set: 3
  rest_time: 45
Inverted Row:
  reps: 12
  set: 3
  rest_time: 45
Plank Hold:
  reps: 30
  set: 3
  rest_time: 30
Forward Lunge:
  reps: 12
  set: 3
  rest_time: 45



In [4]:
def read_yaml_file(file_path: str) -> dict:
    try:
        with open(file_path, "rb") as yaml_file:
            return yaml.safe_load(yaml_file)

    except Exception as e:
        raise  e

In [5]:
read_yaml_file(ymal_str)

OSError: [Errno 22] Invalid argument: 'Standard PushUp:\n  reps: 12\n  set: 3\n  rest_time: 45\nStandard Squart:\n  reps: 12\n  sets: 3\n  rest_time: 45\nHollow Body Hold:\n  reps: 30\n  set: 3\n  rest_time: 45\nInverted Row:\n  reps: 12\n  set: 3\n  rest_time: 45\nPlank Hold:\n  reps: 30\n  set: 3\n  rest_time: 30\nForward Lunge:\n  reps: 12\n  set: 3\n  rest_time: 45\n'

In [1]:
import datetime

date=datetime.datetime(2002,2,21)

print(date.strftime("%a"))

Thu


In [1]:
print(datetime.datetime.now().strftime("%A"))

NameError: name 'datetime' is not defined

In [ ]:
hi={"sajan":12,
    "santosh":15}

a=list(hi.keys())
s,_=a
print(a)



['sajan', 'santosh']


In [1]:

fullbody_moderate_intensity={"Standard PushUp":{"reps":12,"set":3,"rest_time":45},
                            "Standard Squart":{"reps":12,"sets":3,"rest_time":45},
                            "Hollow Body Hold":{"reps":30,"set":3,"rest_time":45},
                            "Inverted Row":{"reps":12,"set":3,"rest_time":45},
                            "Plank Hold":{"reps":30,"set":3,"rest_time":30},
                            "Forward Lunge":{"reps":12,"set":3,"rest_time":45}}



workout_type=["Standard PushUp","Standard Squart","Hollow Body Hold","Inverted Row","Plank Hold","Forward Lunge"]



In [7]:
fullbody_moderate_intensity.get("Standard PushUp")["reps"]=100

In [ ]:
for index , i in enumerate(fullbody_moderate_intensity):
    print

0 --Standard PushUp
1 --Standard Squart
2 --Hollow Body Hold
3 --Inverted Row
4 --Plank Hold
5 --Forward Lunge


In [13]:
print(list(enumerate(fullbody_moderate_intensity)))

[(0, 'Standard PushUp'), (1, 'Standard Squart'), (2, 'Hollow Body Hold'), (3, 'Inverted Row'), (4, 'Plank Hold'), (5, 'Forward Lunge')]


In [ ]:
sajan={"reps":1,"set":3,"rest_time":2},{"reps":4,"set":5,"rest_time":6},{"reps":7,"set":8,"rest_time":9},{"reps":12"set":11,"rest_time":12}]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3144726174.py, line 1)

In [54]:
hi=[{"i":"sajan"},{"i":"suman"},{"i":"sanjana"}]

In [48]:
names=[]
for i in range(len(hi)):
    name=list(hi[i])[0]
    names.append(name)
    
for i,name in enumerate(name):
    print(i,names)

0 ['hi', 'hello', 'bye']
1 ['hi', 'hello', 'bye']
2 ['hi', 'hello', 'bye']


In [3]:
import pandas as pd

hi=[{"i":"sajan"},{"i":"suman"},{"i":"sanjana"}]

df=pd.DataFrame(data=hi)
print(df)

         i
0    sajan
1    suman
2  sanjana
